# **Cuaderno de ETL: De Datos Abiertos a un Modelo de Estrella**

**Objetivo:** Tomar un conjunto de datos públicos sobre estadísticas de educación en Colombia y transformarlo en un modelo dimensional de estrella, listo para ser analizado con herramientas de Business Intelligence.

**Fuente de Datos:** [MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA](https://www.datos.gov.co/Educaci-n/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA/nudc-7mev/about_data)

**Nuestro Modelo de Estrella a Construir:**

* **Tabla de Hechos (Fact_Matriculas):**
    * `id_tiempo` (FK)
    * `id_geografia` (FK)
    * `total_matriculados` (Métrica)
    ...

* **Tablas de Dimensiones:**
    * `Dim_Tiempo` (año)
    * `Dim_Geografia` (departamento, municipio)

¡Manos a la obra!

In [41]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("✅ Librerías importadas.")

# ===================================================================
# PASO 2: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente, lo que es más eficiente que bajar un CSV.
# Aumentamos el límite para traer más filas (ajusta si es necesario).
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla (ej: 404)
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame() # Creamos un dataframe vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()

✅ Librerías importadas.
📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
✅ ¡Extracción exitosa! Se cargaron 14585 filas.


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [42]:
df_raw

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14580,2011,5036,Angelópolis,5,Antioquia,3758,Antioquia (ETC),1707,78.85,78.9,...,3.61,9.5,7.32,0.71,0,0.7,1.08,0,19.57,100
14581,2011,5034,Andes,5,Antioquia,3758,Antioquia (ETC),10244,84.45,84.5,...,0.58,0.04,2.69,5.41,0.73,5.53,6.9,4.11,24.43,93.44
14582,2011,5031,Amalfi,5,Antioquia,3758,Antioquia (ETC),5552,97.71,97.7,...,0,0,0,NaN,0.83,NaN,9.93,4.47,20.01,53.45
14583,2011,5030,Amagá,5,Antioquia,3758,Antioquia (ETC),6631,78.65,78.7,...,6.73,14.46,7.45,0.42,0,0.24,0.91,0,25.05,83.33


## **3. Transformación (Transform)**

Esta es la fase más importante. Aquí limpiamos los datos crudos y los moldeamos para que encajen en nuestro modelo de estrella.

**Pasos:**
1.  **Limpieza y Preparación:** Convertiremos las columnas a los tipos de datos correctos y manejaremos valores faltantes. La columna `matricula` es nuestra métrica principal.
2.  **Creación de Dimensiones:** A partir del DataFrame limpio, crearemos una tabla (DataFrame) para cada dimensión, asegurándonos de que no tengan filas duplicadas y asignando una **llave subrogada** (un ID numérico único).
3.  **Creación de la Tabla de Hechos:** Construiremos la tabla de hechos, que contendrá nuestra métrica (`total_matriculados`) y las llaves foráneas que la conectan a cada dimensión.

## Preguntas

1. Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

2. ¿Cómo compararía el rendimiento educativo por municipios?

3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

Este ejercicio se entrega en un archivo Jupyter Notebook (.ipynb) que contenga el código necesario para realizar las consultas en SQL y que previamente haya creado la bodega de datos con un modelo dimensional adecuado.

## **Desarrollo del Ejercicio**

# Análisis del Sistema Educativo Colombiano
### Construcción de una Bodega de Datos con Modelo Dimensional Tipo Estrella
**Autor:** Jheisson Andrés Pérez González  
**Objetivo:** Construir una bodega de datos usando un modelo dimensional adecuado que permita responder preguntas clave del sistema educativo mediante consultas en SQL.


## **1. Limpieza y Preparación de la Data**

En esta parte el objetivo es dejar los datos en condiciones óptimas para su análisis y posterior carga a una estructura dimensional de acuerdo a la necesidad. A continuación se describen los pasos realizados:

### 1.1 Renombrar Variables

In [43]:
renombrar_columnas = {
    "a_o": "año",
    "c_digo_municipio": "codigo_municipio",
    "c_digo_departamento": "codigo_departamento",
    "c_digo_etc": "codigo_etc",
    "etc": "departamento_etc",
    "poblaci_n_5_16": "poblacion",
    "tasa_matriculaci_n_5_16": "tasa_matriculacion",
    "cobertura_neta_transici_n": "cobertura_neta_transicion",
    "cobertura_bruta_transici_n": "cobertura_bruta_transicion",
    "deserci_n": "desercion",
    "deserci_n_transici_n": "desercion_transicion",
    "deserci_n_primaria": "desercion_primaria",
    "deserci_n_secundaria": "desercion_secundaria",
    "deserci_n_media": "desercion_media",
    "aprobaci_n": "aprobacion",
    "aprobaci_n_transici_n": "aprobacion_transicion",
    "aprobaci_n_primaria": "aprobacion_primaria",
    "aprobaci_n_secundaria": "aprobacion_secundaria",
    "aprobaci_n_media": "aprobacion_media",
    "reprobaci_n": "reprobacion",
    "reprobaci_n_transici_n": "reprobacion_transicion",
    "reprobaci_n_primaria": "reprobacion_primaria",
    "reprobaci_n_secundaria": "reprobacion_secundaria",
    "reprobaci_n_media": "reprobacion_media",
    "repitencia_transici_n": "repitencia_transicion",
    "tama_o_promedio_de_grupo": "tamaño_promedio_de_grupo"
}

df_raw.rename(columns=renombrar_columnas, inplace=True)

print("Columnas renombradas en df_raw:")
print(df_raw.columns.tolist())


Columnas renombradas en df_raw:
['año', 'codigo_municipio', 'municipio', 'codigo_departamento', 'departamento', 'codigo_etc', 'departamento_etc', 'poblacion', 'tasa_matriculacion', 'cobertura_neta', 'cobertura_neta_transicion', 'cobertura_neta_primaria', 'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transicion', 'cobertura_bruta_primaria', 'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'desercion', 'desercion_transicion', 'desercion_primaria', 'desercion_secundaria', 'desercion_media', 'aprobacion', 'aprobacion_transicion', 'aprobacion_primaria', 'aprobacion_secundaria', 'aprobacion_media', 'reprobacion', 'reprobacion_transicion', 'reprobacion_primaria', 'reprobacion_secundaria', 'reprobacion_media', 'repitencia', 'repitencia_transicion', 'repitencia_primaria', 'repitencia_secundaria', 'repitencia_media', 'tamaño_promedio_de_grupo', 'sedes_conectadas_a_internet']


Se estandarizaron los nombres de las columnas para mejorar la legibilidad y consistencia. Por ejemplo, a_o se renombró como año, y poblaci_n_5_16 como poblacion y así sucesivamente.

### 1.2 Cambio del Tipo de Variable

In [44]:
# Lista de variables a convertir a numérico
columnas_numericas = [
    'año',
    'poblacion',
    'tasa_matriculacion',
    'cobertura_neta',
    'cobertura_neta_transicion',
    'cobertura_neta_primaria',
    'cobertura_neta_secundaria',
    'cobertura_neta_media',
    'cobertura_bruta',
    'cobertura_bruta_transicion',
    'cobertura_bruta_primaria',
    'cobertura_bruta_secundaria',
    'cobertura_bruta_media',
    'desercion',
    'desercion_transicion',
    'desercion_primaria',
    'desercion_secundaria',
    'desercion_media',
    'aprobacion',
    'aprobacion_transicion',
    'aprobacion_primaria',
    'aprobacion_secundaria',
    'aprobacion_media',
    'reprobacion',
    'reprobacion_transicion',
    'reprobacion_primaria',
    'reprobacion_secundaria',
    'reprobacion_media',
    'repitencia',
    'repitencia_transicion',
    'repitencia_primaria',
    'repitencia_secundaria',
    'repitencia_media',
    'tamaño_promedio_de_grupo',
    'sedes_conectadas_a_internet'
]


for col in columnas_numericas:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')


Las variables fueron convertidas a sus tipos apropiados:

+ Variables numéricas como matricula, poblacion, aprobacion, tasa_matriculacion se transformaron a float.

+ Variables clave como año, codigo_municipio, codigo_departamento se convirtieron a int.

+ Variables categóricas como departamento se convirtieron a str.

### 1.3 Valores Faltantes

In [45]:
# Mostrar porcentaje de valores nulos por columna
nulos = df_raw.isna().mean().sort_values(ascending=False) * 100
print("Porcentaje de nulos por columna:")
print(nulos)

Porcentaje de nulos por columna:
tamaño_promedio_de_grupo       48.083648
sedes_conectadas_a_internet    46.739801
desercion_transicion            6.191292
desercion_media                 5.032568
desercion_secundaria            1.851217
desercion_primaria              1.659239
repitencia_transicion           1.090161
repitencia_secundaria           1.042167
repitencia_primaria             1.014741
reprobacion_media               0.994172
repitencia                      0.980459
desercion                       0.973603
repitencia_media                0.953034
cobertura_bruta_media           0.870758
tasa_matriculacion              0.788481
cobertura_neta                  0.761056
reprobacion_secundaria          0.726774
aprobacion_media                0.692492
cobertura_bruta_transicion      0.665067
reprobacion_primaria            0.665067
cobertura_neta_secundaria       0.644498
reprobacion_transicion          0.637641
aprobacion_transicion           0.637641
cobertura_neta_media    

Teniendo en cuenta el porcentaje de valores faltantes de las variables tamaño promedio de grupo y sedes conectadas a internet, se decidió quitarlas del dataframe ya que si se necesitan en algún momento se pueden calcular con las demás variables disponibles; en cuanto a las demás, se optó por hacer imputación con el valor de la media

### 1.4 Imputación de datos faltantes 

In [46]:
variables_a_imputar_media = [
    'desercion_transicion',
    'desercion_media',
    'desercion_secundaria',
    'desercion_primaria',
    'repitencia_transicion',
    'repitencia_secundaria',
    'repitencia_primaria',
    'reprobacion_media',
    'repitencia',
    'desercion',
    'repitencia_media',
    'cobertura_bruta_media',
    'tasa_matriculacion',
    'cobertura_neta',
    'reprobacion_secundaria',
    'aprobacion_media',
    'cobertura_bruta_transicion',
    'reprobacion_primaria',
    'cobertura_neta_secundaria',
    'reprobacion_transicion',
    'aprobacion_transicion',
    'cobertura_neta_media',
    'cobertura_neta_primaria',
    'cobertura_bruta_secundaria',
    'reprobacion',
    'cobertura_bruta_primaria',
    'cobertura_bruta',
    'aprobacion_secundaria',
    'cobertura_neta_transicion',
    'aprobacion',
    'aprobacion_primaria',
    'poblacion'
]


for col in variables_a_imputar_media:
    media = df_raw[col].mean()
    df_raw[col].fillna(media, inplace=True)
    print(f"Imputada columna: {col} con media = {media:.4f}")


Imputada columna: desercion_transicion con media = 3.4868
Imputada columna: desercion_media con media = 3.4663
Imputada columna: desercion_secundaria con media = 4.5779
Imputada columna: desercion_primaria con media = 2.7567
Imputada columna: repitencia_transicion con media = 0.9432
Imputada columna: repitencia_secundaria con media = 4.3265
Imputada columna: repitencia_primaria con media = 3.1695
Imputada columna: reprobacion_media con media = 4.1308
Imputada columna: repitencia con media = 3.2995
Imputada columna: desercion con media = 3.4899
Imputada columna: repitencia_media con media = 1.6979
Imputada columna: cobertura_bruta_media con media = 75.8558
Imputada columna: tasa_matriculacion con media = 84.9719
Imputada columna: cobertura_neta con media = 85.5532
Imputada columna: reprobacion_secundaria con media = 6.8090
Imputada columna: aprobacion_media con media = 92.1405
Imputada columna: cobertura_bruta_transicion con media = 87.1303
Imputada columna: reprobacion_primaria con med

C:\Users\ANDRES PEREZ\AppData\Local\Temp\ipykernel_15200\2192922434.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_raw[col].fillna(media, inplace=True)


### 1.5 Cargue Base DIVIPOLA

In [47]:
import pandas as pd

url_divipola = "https://www.datos.gov.co/resource/gdxc-w37w.json?$limit=5000"
divipola = pd.read_json(url_divipola)

# Vista previa
divipola.head()

,cod_dpto,dpto,cod_mpio,nom_mpio,tipo_municipio,longitud,latitud
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631"
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio,"-75,428739","5,789315"
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio,"-76,064304","6,632282"
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio,"-75,141346","6,376061"
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio,"-75,702188","6,038708"


### 1.6 Renombrar Variables de DIVIPOLA

In [48]:
# Renombrar columnas de DIVIPOLA para hacer match con mis dimensiones
divipola = divipola.rename(columns={
    'cod_dpto': 'codigo_departamento',
    'dpto': 'departamento',
    'cod_mpio': 'codigo_municipio',
    'nom_mpio': 'municipio',
    'tipo_municipio': 'categoria',
    'longitud': 'longitud',
    'latitud': 'latitud'
})

La única modificación que se tuvo que hacer para esta base fue renombrar las variables anteriormente descritas para que coincidieran con las bases a trabajar.

## 2. **Creación de Dimensiones**

In [49]:
# Quitar espacios y convertir a numérico forzadamente
df_raw['codigo_departamento'] = pd.to_numeric(df_raw['codigo_departamento'], errors='coerce')
df_raw['codigo_municipio'] = pd.to_numeric(df_raw['codigo_municipio'], errors='coerce')
df_raw['codigo_etc'] = pd.to_numeric(df_raw['codigo_etc'], errors='coerce')

# Eliminar filas con valores inválidos
df_raw = df_raw.dropna(subset=['codigo_departamento', 'codigo_municipio', 'codigo_etc'])

# Convertir a int
df_raw['codigo_departamento'] = df_raw['codigo_departamento'].astype(int)
df_raw['codigo_municipio'] = df_raw['codigo_municipio'].astype(int)
df_raw['codigo_etc'] = df_raw['codigo_etc'].astype(int)


In [50]:
# 🔹 2. Dimensión de tiempo
dim_tiempo = df_raw[['año']].drop_duplicates().copy()
dim_tiempo['id_tiempo'] = dim_tiempo.reset_index().index + 1
dim_tiempo = dim_tiempo[['id_tiempo', 'año']]

# 🔹 3. Dimensión de departamento
dim_departamento = df_raw[['codigo_departamento', 'departamento']].drop_duplicates()
dim_departamento = dim_departamento.merge(
    divipola[['codigo_departamento', 'departamento']].drop_duplicates(),
    on='codigo_departamento', how='left', suffixes=('', '_divipola')
)
dim_departamento['id_departamento'] = dim_departamento.reset_index().index + 1
dim_departamento = dim_departamento[['id_departamento', 'codigo_departamento', 'departamento']]

# 🔹 4. Dimensión de municipio
dim_municipio = df_raw[['codigo_municipio', 'codigo_departamento']].drop_duplicates()
dim_municipio = dim_municipio.merge(
    divipola[['codigo_municipio', 'municipio', 'categoria', 'latitud', 'longitud']],
    on='codigo_municipio', how='left'
)
dim_municipio['id_municipio'] = dim_municipio.reset_index().index + 1
dim_municipio = dim_municipio[['id_municipio', 'codigo_municipio', 'codigo_departamento', 
                               'municipio', 'categoria', 'latitud', 'longitud']]

# 🔹 5. Dimensión ETC (Entidad Territorial Certificada)
dim_etc = df_raw[['codigo_etc']].drop_duplicates()
dim_etc['id_etc'] = dim_etc.reset_index().index + 1
dim_etc = dim_etc[['id_etc', 'codigo_etc']]

# 🔹 6. Nueva dimensión geográfica
dim_geo = divipola[['codigo_departamento', 'departamento',
                    'codigo_municipio', 'municipio',
                    'categoria', 'latitud', 'longitud']].drop_duplicates()
dim_geo['id_geo'] = dim_geo.reset_index().index + 1
dim_geo = dim_geo[['id_geo', 'codigo_departamento', 'departamento',
                   'codigo_municipio', 'municipio',
                   'categoria', 'latitud', 'longitud']]

In [51]:

# Revisar estructura
print(dim_tiempo.head())
print(dim_departamento.head())
print(dim_municipio.head())
print(dim_etc.head())
print(dim_geo.head())


      id_tiempo   año
0             1  2023
1121          2  2022
2242          3  2021
3364          4  2020
4486          5  2019
   id_departamento  codigo_departamento departamento
0                1                    5    Antioquia
1                2                   95     Guaviare
2                3                   97       Vaupés
3                4                   99      Vichada
4                5                   94      Guainía
   id_municipio  codigo_municipio  codigo_departamento   municipio  categoria  \
0             1              5004                    5    ABRIAQUÍ  Municipio   
1             2             95025                   95  EL RETORNO  Municipio   
2             3             95200                   95  MIRAFLORES  Municipio   
3             4             97001                   97        MITÚ  Municipio   
4             5             97161                   97      CARURÚ  Municipio   

    latitud    longitud  
0  6,632282  -76,064304  
1  2,330164

## **3. Creación Tablas de Hechos**

### 3.1 Tabla de Métricas

In [52]:
metricas = [
    'poblacion',
    'tasa_matriculacion',
    'cobertura_neta',
    'cobertura_neta_transicion',
    'cobertura_neta_primaria',
    'cobertura_neta_secundaria',
    'cobertura_neta_media',
    'cobertura_bruta',
    'cobertura_bruta_transicion',
    'cobertura_bruta_primaria',
    'cobertura_bruta_secundaria',
    'cobertura_bruta_media',
    'desercion',
    'desercion_transicion',
    'desercion_primaria',
    'desercion_secundaria',
    'desercion_media',
    'aprobacion',
    'aprobacion_transicion',
    'aprobacion_primaria',
    'aprobacion_secundaria',
    'aprobacion_media',
    'reprobacion',
    'reprobacion_transicion',
    'reprobacion_primaria',
    'reprobacion_secundaria',
    'reprobacion_media',
    'repitencia',
    'repitencia_transicion',
    'repitencia_primaria',
    'repitencia_secundaria',
    'repitencia_media'
]


In [53]:
# Crear tabla de hechos con claves foráneas y métricas
df_hechos = df_raw.copy()

# Uniones con dimensiones principales
df_hechos = df_hechos.merge(dim_tiempo, on='año', how='left')
df_hechos = df_hechos.merge(dim_departamento, on=['codigo_departamento', 'departamento'], how='left')
df_hechos = df_hechos.merge(dim_municipio, on=['codigo_municipio', 'codigo_departamento'], how='left')
df_hechos = df_hechos.merge(dim_etc, on='codigo_etc', how='left')

# 🔹 Unión con dim_geo para obtener id_geo
df_hechos = df_hechos.merge(dim_geo[['codigo_municipio', 'id_geo']], on='codigo_municipio', how='left')

# Selección final de claves foráneas y métricas
df_hechos = df_hechos[[
    'id_tiempo', 'id_departamento', 'id_municipio', 'id_etc', 'id_geo'
] + metricas]

# Resetear índice
df_hechos.reset_index(drop=True, inplace=True)



### 3.2 Creación base de datos y almacenamiento de las tablas

In [54]:
import sqlite3

# Crear conexión a base SQLite local (creará el archivo si no existe)
conn = sqlite3.connect("Modelos_dimensionales.db")

# Guardar las dimensiones
dim_tiempo.to_sql("dim_tiempo", conn, index=False, if_exists="replace")
dim_departamento.to_sql("dim_departamento", conn, index=False, if_exists="replace")
dim_municipio.to_sql("dim_municipio", conn, index=False, if_exists="replace")
dim_etc.to_sql("dim_etc", conn, index=False, if_exists="replace")
dim_geo.to_sql("dim_geo", conn, index=False, if_exists="replace")  # ✅ nueva dimensión

# Guardar tabla de hechos con id_geo incluida
df_hechos.to_sql("hechos_matricula", conn, index=False, if_exists="replace")  # ya contiene id_geo

# Cerrar conexión (opcional)
conn.close()

print("✅ Tablas guardadas correctamente en 'Modelos_dimensionales.db'")



✅ Tablas guardadas correctamente en 'Modelos_dimensionales.db'


In [55]:
import sqlite3

# Reabrir la conexión
conn = sqlite3.connect("Modelos_dimensionales.db")

In [56]:
def ejecutar_sql(query, conexion=conn):
  """
  Toma una consulta SQL, la ejecuta y devuelve el resultado
  en una tabla de pandas para una visualización clara.
  """
  df = pd.read_sql_query(query, conexion)
  # Usamos display() porque en Colab presenta las tablas de forma más elegante.
  display(df)

## **4. Respondiendo las Preguntas Planteadas...**

### 4.1 Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

In [57]:
ejecutar_sql("""
SELECT 
    g.municipio,
    ROUND(AVG(h.tasa_matriculacion), 2) AS escolaridad_promedio
FROM hechos_matricula h
JOIN dim_geo g ON h.id_geo = g.id_geo
GROUP BY g.municipio
ORDER BY escolaridad_promedio DESC;
""")



,municipio,escolaridad_promedio
0,COTA,161.88
1,LA CALERA,144.08
2,SALENTO,139.27
3,TENJO,134.69
4,FÚQUENE,128.59
...,...,...
1031,FRANCISCO PIZARRO,45.78
1032,MIRITÍ - PARANÁ,45.55
1033,EL ENCANTO,45.42
1034,MAGÜÍ,43.25


### 4.2 ¿Cómo compararía el rendimiento educativo por municipios?

In [58]:
ejecutar_sql("""
SELECT 
    g.municipio,
    ROUND(AVG(h.aprobacion), 2) AS promedio_aprobacion,
    ROUND(AVG(h.reprobacion), 2) AS promedio_reprobacion,
    ROUND(AVG(h.desercion), 2) AS promedio_desercion
FROM hechos_matricula h
JOIN dim_geo g ON h.id_geo = g.id_geo
GROUP BY g.municipio
ORDER BY promedio_aprobacion DESC;
""")



,municipio,promedio_aprobacion,promedio_reprobacion,promedio_desercion
0,PUERTO ARICA,98.97,0.03,1.81
1,TIBIRITA,98.42,0.43,1.16
2,LA PALMA,98.28,0.20,1.52
3,GUATAVITA,98.27,0.27,1.45
4,MIRITÍ - PARANÁ,98.21,0.35,1.98
...,...,...,...,...
1031,BARRANCOMINAS,78.20,12.11,5.30
1032,CUMARIBO,77.20,14.32,7.49
1033,INÍRIDA,77.08,12.16,7.30
1034,CACAHUAL,75.37,11.36,7.03


### 4.3 ¿Qué departamentos son los que mejor cobertura tienen? ¿Se puede calcular con SQL?

In [59]:
ejecutar_sql("""
SELECT 
    d.departamento,
    ROUND(AVG(h.cobertura_neta), 2) AS promedio_cobertura_neta,
    ROUND(AVG(h.cobertura_bruta), 2) AS promedio_cobertura_bruta
FROM hechos_matricula h
JOIN dim_departamento d ON h.id_departamento = d.id_departamento
GROUP BY d.departamento
ORDER BY promedio_cobertura_neta DESC;
""")



,departamento,promedio_cobertura_neta,promedio_cobertura_bruta
0,Bogotá D.C.,95.89,104.74
1,Quindio,94.58,109.72
2,Sucre,93.85,111.07
3,Cesar,93.73,108.44
4,Magdalena,93.26,113.81
5,Meta,90.63,102.39
6,Tolima,89.24,99.66
7,"Bogotá, D.C.",88.96,95.55
8,Cundinamarca,88.54,100.22
9,Casanare,88.52,98.67


## 5. **¿Por qué Escoger el esquema estrella en esta ocasión?**

El modelo dimensional tipo estrella fue seleccionado para estructurar la bodega de datos del sistema educativo debido a su simplicidad estructural, eficiencia en consultas y adecuada alineación con los objetivos analíticos del proyecto. Su diseño, que consiste en una tabla central de hechos conectada a múltiples dimensiones independientes, facilita la comprensión del modelo y simplifica la escritura de consultas SQL. 

Esta estructura también permite realizar análisis eficientes al evitar uniones complejas, optimizando así el rendimiento en consultas agregadas y segmentadas por variables como año, municipio, departamento y entidad territorial certificada. 

Dado que el objetivo en este caso es analizar y visualizar el comportamiento del sistema educativo a partir de métricas como tasa de matriculación, aprobación, reprobación, deserción y cobertura, este modelo estrella se adapta perfectamente. Esta elección también garantiza una alta compatibilidad con herramientas de análisis y visualización como Power BI, Tableau o Jupyter Notebooks, y ofrece escalabilidad suficiente para el volumen actual de datos (~50.000 registros), con posibilidad de crecimiento futuro sin requerir rediseños estructurales profundos. Por estas razones, el modelo estrella representa una solución robusta, clara y eficiente para responder a las necesidades del análisis educativo en este caso de estudio.

## 6. **Conclusiones**

- Se construyó correctamente una bodega de datos a partir de fuentes públicas como el Ministerio de Educación y DIVIPOLA.
- Se aplicó un modelo dimensional tipo estrella, el cual permitió organizar de forma clara y eficiente las métricas educativas junto con sus dimensiones contextuales.
- Las consultas SQL implementadas permiten responder preguntas clave como el porcentaje de escolaridad por municipio, el rendimiento educativo regional y la cobertura departamental.
